# Zipline 管道

### 简介

在任何一个交易日，整个股票池都包含成千上万个证券。通常，你并不想投资所有的股票，而是只选择一部分股票。例如，你可能只想投资 10 天平均收盘价为 10 美元及以下的股票。或者，你可能只想投资根据某个因子排名前 500 的证券。

为了避免花大量时间挖掘数据，并只选择你想要的股票，人们通常会使用**管道**。一般而言，管道是一个占位符，其中包含一系列根据某些因子过滤数据并加以排名的数据操作。

在此 notebook 中，你将学习如何使用 **Zipline 管道**。Zipline 是由 *Quantopian* 开发的一款开源算术交易模拟器。我们将学习如何使用 Zipline 管道根据因子过滤股票数据。 

### 安装软件包

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

# 通过 Zipline 加载数据

在使用 Zipline 开始构建管道之前，首先需要了解如何将要使用的股票数据加载到 Zipline 中。Zipline 使用**数据包**轻松处理不同的数据源。数据包是一个股价数据、调整数据和资产数据的集合。Zipline 通过数据包预加载运行回溯测试用到的数据和存储未来运行用到的数据。Zipline 默认包含几个数据包，还可以提取新的数据包。使用数据包的第一步是提取数据。在 Zipline 提取流程中，Zipline 首先会下载数据或从本地机器加载数据文件。然后，将数据传递给一组写入器对象，这些对象会将原始数据转换成 Zipline 的内部格式（股价数据为`bcolz`，拆分/合并/股息数据为 `SQLite` ），这些格式经过优化，可以提高运行速度。新的数据会写入 Zipline 能找到的标准位置。默认情况下，新数据会写入 `ZIPLINE_ROOT/data/<bundle>` 的子目录中，其中 `<bundle>` 是为提取的数据包设定的名称，子目录的名称中包含当前日期。这样 Zipline 便能查看更早的数据并对更早的数据运行回溯测试。使用提取的旧数据进行回溯测试，日后更容易重现回溯结果。

在此 notebook 中，我们将使用 **Quotemedia** 的股票数据。在 Udacity workspace 中，你将发现 Quotemedia 的股票数据已经提取到 Zipline 中。所以，在以下代码中，我们将使用 Zipline 的 `bundles.load()` 函数加载之前提取的 Quotemedia 股票数据。为了使用 `bundles.load()` 函数，我们需要先执行几步操作。首先，我们需要指定之前提取的数据包的名称。在此练习中，Quotemedia 数据包的名称为 `eod-quotemedia`：

In [ ]:
# Specify the bundle name
bundle_name = 'eod-quotemedia'

其次，我们需要使用 `bundles.register()` 函数注册数据包及其提取函数。提取函数负责将数据加载到内存中，并将其传递给 Zipline 提供的一组写入器对象，以将数据转换为 Zipline 的内部格式。因为原始 Quotemedia 数据包含在  `.csv` 文件中，所以我们将使用 `csvdir_equities()` 函数为 Quotemedia 数据包生成提取函数。此外，因为 Quotemedia 的 `.csv` 文件包含每日股票数据，所以我们将提取函数的日期范围设为 `daily`。

In [ ]:
from zipline.data import bundles
from zipline.data.bundles.csvdir import csvdir_equities

# Create an ingest function 
ingest_func = csvdir_equities(['daily'], bundle_name)

# Register the data bundle and its ingest function
bundles.register(bundle_name, ingest_func);

注册数据包和提取函数后，我们可以使用 `bundles.load()` 函数加载数据。因为此函数会加载之前提取的数据，所以我们需要将 `ZIPLINE_ROOT` 设为最近提取数据所在的路径。最近的数据位于 `cwd/../../data/project_4_eod/` 目录下，其中 `cwd` 是当前的工作目录。我们将使用 `os.environ[]` 命令指定该位置。

In [ ]:
import os

# Set environment variable 'ZIPLINE_ROOT' to the path where the most recent data is located
os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '..', '..', 'data', 'project_4_eod')

# Load the data bundle
bundle_data = bundles.load(bundle_name)

# 构建空管道

加载数据后，就可以开始构建 Zipline 管道了。首先，使用 Zipline 的 `Pipeline` 类创建一个空的 Pipeline 对象。Pipeline 对象表示由管道引擎编译和执行的一组已命名表达式。`Pipeline(columns=None, screen=None)` 类接受两个可选参数 `columns` 和 `screen`。`columns` 参数是一个字典，表示要使用的初始列。`screen` 参数用于设置一个筛选器来排除不需要的数据。

在以下代码中，我们将使用 Zipline 的内置 `.AverageDollarVolume()` 类为管道创建一个 `screen`。我们将使用 `.AverageDollarVolume()` 类为股票池中的每支股票生成一个 60 天平均收盘价。然后使用 `.top(10)` 属性每天从股票池中过滤出前 10 项资产。所以，此筛选器将充当过滤器，每天从股票池中排除一些数据。平均交易量可以作为第一道屏障，有效避免选择不流动的资产。

In [ ]:
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume

# Create a screen for our Pipeline
universe = AverageDollarVolume(window_length = 60).top(10)

# Create an empty Pipeline with the given screen
pipeline = Pipeline(screen = universe)

在上述代码中，我们将 Pipeline 对象命名为 `pipeline`，以便后面进行计算时能够引用它。注意，Pipeline 是一个对象，表示我们希望每天都执行的运算。新创建的管道是空的。所以它不知道如何执行任何计算，并且返回输出的话，它不会生成任何值。在以下部分，我们将了解如何向管道提供要计算的表达式。

# 因子和过滤器

上面使用的 `.AverageDollarVolume()` 类属于因子。在此部分，我们将了解管道可以表达的两种计算形式：**因子**和**过滤器**。通常，因子和过滤器表示针对某一时刻的资产生成值的函数，但是它们生成的值类型有区别。下面先看看因子。


### 因子

通常，**因子**是指关于特定时间的资产数值函数。证券的最近价格就是一个简单的因子示例。给定证券和特定的时间点，最近的价格就是一个数字。证券的 10 天平均交易量也属于因子。因子通常用于向证券赋值，然后与过滤器或其他因子相结合。能够组合多个因子使我们能够轻松地构建各种复杂的自定义因子。例如，如果要构建一个因子来计算另外两个因子的平均值，步骤如以下伪代码所示：
```python
f1 = factor1(...)
f2 = factor2(...)  
average = (f1 + f2) / 2.0  
```

### 过滤器

通常，**过滤器**是指关于特定时间的资产布尔值函数（True 或 False）。表示证券价格是否低于 5 美元的函数就属于过滤器。给定证券和特定的时间点，这个函数的结果将是 **True** 或 **False**。过滤器最常用于选择一组包含在股票池中或从中排除掉的证券。我们通常使用比较运算符创建过滤器，例如 <、<=、!=、==、>、>=。

# 以图表形式查看管道

使用 Zipline 的 Pipeline 类中的属性 `.show_graph()` 可以将管道渲染为有向非循环图 (DAG)。此图是使用 DOT 语言指定的，所以我们需要使用 DOT 图形布局程序来查看渲染的图像。在以下代码中，我们将使用 Graphviz 软件包渲染 `.show_graph()` 属性生成的图形。Graphviz 是一个开源软件包，它可以绘制用 DOT 语言脚本指定的图形。

In [ ]:
import graphviz

# Render the pipeline as a DAG
pipeline.show_graph()

目前管道是空的，只包含一个筛选器。所以，当我们渲染 `pipeline` 时，我们只看到 `screen` 图表：
```python
AverageDollarVolume(window_length = 60).top(10)
```

默认情况下，`.AverageDollarVolume()` 类使用 `USEquityPricing` 数据集来计算平均交易量，该数据集包含每日交易价格和交易量：
```python
average_dollar_volume = np.nansum(close_price * volume, axis=0) / len(close_price)
```

图表的顶部表示 `.AverageDollarVolume()` 类从 `USEquityPricing` 数据集中获取输入（收盘价和交易量）。图表的底部表示输出由表达式 `x_0 <= 10` 确定。此表达式表示我们使用 `.top(10)` 作为 `screen` 中的过滤器。我们将图表中的每个方框称为一项。

# 数据集和数据加载器               

Zipline 的 Pipeline 有一个功能是将股票数据的实际来源与该数据集的抽象描述区分开。所以，Zipline 会针对这些数据集使用 **DataSets** 和 **Loaders**。`DataSets` 是标记值的抽象集合，这些标记值描述了特定数据集的列/类型。而 `loader` 是一个对象，对特定数据集提出请求后，它能够实际获取请求的数据。例如，`USEquityPricing` 数据集使用的加载器是 `USEquityPricingLoader` 类。`USEquityPricingLoader` 类会将加载基准线和调整工作委派给更低级别的子系统，这些子系统知道如何获取股价数据，并且采用 Zipline 默认格式（股价数据为 `bcolz`，拆分/合并/股息数据为 `SQLite`）。正如我们在此 notebook 的开头所看到的，数据包自动将股票数据转换为 `bcolz` 和 `SQLite` 格式。需要注意的是，除了 `USEquityPricing` 数据集之外，`USEquityPricingLoader` 类还可以用于加载其他数据集的 OHLCV 数据。同理，还可以为相同的数据集编写不同的加载器，并使用这些加载器（而不是默认加载器）。Zipline 包含很多其他加载器，你可以利用这些加载器从不同的数据集中加载数据。

在以下代码中，我们将使用 `USEquityPricingLoader(BcolzDailyBarWriter, SQLiteAdjustmentWriter)` 从 `bcolz` 定价数据目录和 `SQLite` 调整数据目录中创建加载器。`BcolzDailyBarWriter` 和 `SQLiteAdjustmentWriter` 确定了定价数据和调整数据的路径。因为我们将使用 Quotemedia 数据包，所以将使用 `bundle_data.equity_daily_bar_reader` 和 `bundle_data.adjustment_reader` 分别作为 `BcolzDailyBarWriter` 和 `SQLiteAdjustmentWriter`。

In [ ]:
from zipline.pipeline.loaders import USEquityPricingLoader

# Set the dataloader
pricing_loader = USEquityPricingLoader(bundle_data.equity_daily_bar_reader, bundle_data.adjustment_reader)

# 管道引擎

Zipline 利用计算引擎来执行管道。在以下代码中，我们将使用 Zipline 的`SimplePipelineEngine()` 类作为执行管道的引擎。`SimplePipelineEngine(get_loader, calendar, asset_finder)` 类会将所选数据加载器与对应的数据集和交易日历关联起来。`get_loader` 参数必须是可调用的函数，向其提供可加载的项，并返回 `PipelineLoader`，用于为管道中的该项检索原始数据。我们将使用上面定义的 `pricing_loader` ，所以创建一个 `choose_loader` 函数并返回 `pricing_loader`。该函数还会检查请求的数据是否对应于 OHLCV 数据，不对应则返回错误。`calendar` 参数必须是一个日期 `DatetimeIndex` 数组，在计算固定的 `start_date` 和 `end_date` 范围时，将这些日期看做交易日。我们将使用纽约证交所使用的交易日。我们将使用 Zipline 的 `get_calendar('NYSE')` 函数检索纽约证交所使用的交易日。然后使用 `.all_sessions` 属性从 `trading_calendar` 中获取 `DatetimeIndex`，并将其传递给 `calendar` 参数。最后，`asset_finder` 参数决定了在任何时间点位于股票数据中的前几位的资产。因为我们将使用 Quotemedia 数据包，所以将此参数设为 `bundle_data.asset_finder`。

In [ ]:
from zipline.utils.calendars import get_calendar
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.engine import SimplePipelineEngine

# Define the function for the get_loader parameter
def choose_loader(column):
    if column not in USEquityPricing.columns:
        raise Exception('Column not in USEquityPricing')
    return pricing_loader

# Set the trading calendar
trading_calendar = get_calendar('NYSE')

# Create a Pipeline engine
engine = SimplePipelineEngine(get_loader = choose_loader,
                              calendar = trading_calendar.all_sessions,
                              asset_finder = bundle_data.asset_finder)

# 运行管道

选择引擎后，我们就可以运行或执行管道了。我们可以使用 `SimplePipelineEngine` 类的 `.run_pipeline()` 属性运行管道。具体而言，`SimplePipelineEngine.run_pipeline(pipeline, start_date, end_date)` 会实现以下算法并执行管道：


1. 构建一个 `pipeline` 中所有项的依赖图。在这一步，图形按拓扑顺序排序，确定了计算各项的顺序。


2. 通过 AssetFinder 查找“lifetimes matrix”，并且对于 `start_date` 和 `end_date` 之间的每个日期，针对每项已知资产包含一个布尔值，表示在该日期是否存在该资产。


3. 按照第 1 步确定的依赖顺序计算每项，将结果缓存到字典中，从而将结果传递给后面的项。


4. 对于每个日期，确定通过 `pipeline` 筛选器的资产数量。所有这些值的和 $N$ 表示输出 Pandas Dataframe 中的总行数，所以为项中的每个因子提前分配长度为 $N$ 的输出数组。


5. 通过将输出缓冲中计算的值复制到相应的行中，填充在第 4 步分配的数组。


6. 将在第 5 步计算的值存储到 Pandas DataFrame 中并返回该 DataFrame。

在以下代码中，使管道运行一天，所以 `start_date` 和 `end_date` 是一样的。然后输出关于 `pipeline_output` 的信息。

In [ ]:
import pandas as pd

# Set the start and end dates
start_date = pd.Timestamp('2016-01-05', tz = 'utc')
end_date = pd.Timestamp('2016-01-05', tz = 'utc')

# Run our pipeline for the given start and end dates
pipeline_output = engine.run_pipeline(pipeline, start_date, end_date)

# We print information about the pipeline output
print('The pipeline output has type:', type(pipeline_output), '\n')

# We print whether the pipeline output is a MultiIndex Dataframe
print('Is the pipeline output a MultiIndex Dataframe:', isinstance(pipeline_output.index, pd.core.index.MultiIndex), '\n')

# If the pipeline output is a MultiIndex Dataframe we print the two levels of the index
if isinstance(pipeline_output.index, pd.core.index.MultiIndex):

    # We print the index level 0
    print('Index Level 0:\n\n', pipeline_output.index.get_level_values(0), '\n')

    # We print the index level 1
    print('Index Level 1:\n\n', pipeline_output.index.get_level_values(1), '\n')

从上面可以看到，`.run_pipeline()` 的返回值是一个 `MultiIndex` Pandas DataFrame，通过管道筛选器的每项资产都对应一行。并且第 0 级索引包含日期，第 1 级索引包含 ticker。通常，在返回的 Pandas DataFrame 中，我们使用  `Pipeline.add()` 添加的每个因子和过滤器也对应一列。目前我们未向管道添加任何因子或过滤器，所以 Pandas Dataframe 没有任何列。在以下部分，我们将了解如何向管道中添加因子和过滤器。

# 获取 Ticker

在上一部分我们发现，通过管道筛选器的股票对应的 ticker 包含在第 1 级索引中。所以，我们可以使用 Pandas `.get_level_values(1).values.tolist()` 方法获取这些股票的 ticker 并将其保存到列表中。

In [ ]:
# Get the values in index level 1 and save them to a list
universe_tickers = pipeline_output.index.get_level_values(1).values.tolist()

# Display the tickers
universe_tickers

# 获取数据

获取通过管道筛选器的股票对应的 ticker 后，我们可以从数据包中获取这些 ticker 的历史股票数据。我们需要使用 Zipline 的 `DataPortal` 类获取历史数据。 `DataPortal` 是 Zipline 模拟所需的所有数据的接口。在以下代码中，我们将创建 `DataPortal` 和 `get_pricing` 函数来获取 ticker 的历史股票数据。

在创建 `DataPortal` 时，我们已经见过以下大部分参数。所以这里就不解释了。唯一的新参数是 `first_trading_day`。 `first_trading_day` 参数是一个  `pd.Timestamp`，表示模拟的首个交易日。我们将首个交易日设为数据包中的首个交易日。要详细了解 `DataPortal` 类，请参阅此 [Zipline 文档](https://www.zipline.io/appendix.html?highlight=dataportal#zipline.data.data_portal.DataPortal)

In [ ]:
from zipline.data.data_portal import DataPortal

# Create a data portal
data_portal = DataPortal(bundle_data.asset_finder,
                         trading_calendar = trading_calendar,
                         first_trading_day = bundle_data.equity_daily_bar_reader.first_trading_day,
                         equity_daily_reader = bundle_data.equity_daily_bar_reader,
                         adjustment_reader = bundle_data.adjustment_reader)

创建 `data_portal` 后，我们将创建一个辅助函数 `get_pricing`，它会针对从 `start_date` 到 `end_date` 之间的日期从 `data_portal` 获取历史数据。`get_pricing` 函数包含多种参数：
```python
def get_pricing(data_portal, trading_calendar, assets, start_date, end_date, field='close')
```

前两个参数 `data_portal` 和 `trading_calendar` 在上面已经解释过。第三个参数 `assets` 是一个 ticker 列表。我们将使用管道输出中的 ticker，即 `universe_tickers`。第四个参数是指定 `start_date` 和 `end_date` 的字符串。函数会将这两个字符串转换为具有自定义工作日频率的时间戳。最后一个参数 `field` 是一个字符串，表示返回哪个字段。我们想要获得收盘价，所以设为 `field='close`。

该函数使用 `DataPortal` 类的 `.get_history_window()` 属性返回历史股价数据。该属性返回一个 Pandas Dataframe，其中包含请求的历史窗口，并且数据已完全调整。`bar_count` 参数是一个整数，表示返回的天数。天数确定了返回的 Dataframe 的行数。`frequency` 和 `data_frequency` 参数都是字符串，表示查询数据的频率，即按 `daily` 或 `minute` 查询数据。

In [ ]:
def get_pricing(data_portal, trading_calendar, assets, start_date, end_date, field='close'):
    
    # Set the given start and end dates to Timestamps. The frequency string C is used to
    # indicate that a CustomBusinessDay DateOffset is used
    end_dt = pd.Timestamp(end_date, tz='UTC', freq='C')
    start_dt = pd.Timestamp(start_date, tz='UTC', freq='C')

    # Get the locations of the start and end dates
    end_loc = trading_calendar.closes.index.get_loc(end_dt)
    start_loc = trading_calendar.closes.index.get_loc(start_dt)

    # return the historical data for the given window
    return data_portal.get_history_window(assets=assets, end_dt=end_dt, bar_count=end_loc - start_loc,
                                          frequency='1d',
                                          field=field,
                                          data_frequency='daily')

# Get the historical data for the given window
historical_data = get_pricing(data_portal, trading_calendar, universe_tickers,
                              start_date='2011-01-05', end_date='2016-01-05')
# Display the historical data
historical_data

# 数据对齐

当管道返回日期 `2016-01-07` 时，包含的是在 `2016-01-07` **开盘**前已知的数据。因此，如果你要获取每天最新的已知值，它将返回前一天的收盘价，并添加日期标签 `2016-01-07`。所有因子值都假定利用在此时间点之前已知的数据在标记的日期开盘前运行。

# 添加因子和过滤器

你已经知道如何构建和执行管道，在此部分，我们将了解如何向管道中添加因子和过滤器。这些因子和过滤器决定了我们希望管道每天执行哪些运算。

我们可以使用 `Pipeline` 类的 `.add(column, name)` 方法添加因子和过滤器。`column` 参数表示要添加到管道中的因子或过滤器。`name` 参数是一个字符串，决定了该因子或过滤器在输出 Pandas Dataframe 中对应的列名称。正如之前提到的，每个因子和过滤器将在管道的输出 Dataframe 中以列的形式出现。我们先向管道中添加因子。

### 因子

在以下代码中，我们将使用 Zipline 的内置 `SimpleMovingAverage` 因子创建一个因子，用来计算证券的 15 天平均收盘价。 然后将此因子添加到管道中，并使用 `.show_graph()` 显示添加该因子后的管道图表。

In [ ]:
from zipline.pipeline.factors import SimpleMovingAverage

# Create a factor that computes the 15-day mean closing price of securities
mean_close_15 = SimpleMovingAverage(inputs = [USEquityPricing.close], window_length = 15)

# Add the factor to our pipeline
pipeline.add(mean_close_15, '15 Day MCP')

# Render the pipeline as a DAG
pipeline.show_graph()

在上面的图表中，可以清晰地看到我们添加的因子。现在我们可以再次运行管道并看看输出。像之前那样运行管道。

In [ ]:
# Set starting and end dates
start_date = pd.Timestamp('2014-01-06', tz='utc')
end_date = pd.Timestamp('2016-01-05', tz='utc')

# Run our pipeline for the given start and end dates
output = engine.run_pipeline(pipeline, start_date, end_date)

# Display the pipeline output
output.head()

可以看到，现在输出 dataframe 包含一个名为 `15 Day MCP` 的列，即我们之前为因子设定的名称。这个管道输出 dataframe 包含通过 `screen` 的证券的 15 天平均收盘价。

### 过滤器

我们像之前一样创建过滤器并添加到管道中。在以下代码中，我们创建一个过滤器，每当 15 天平均收盘价高于 100 美元时，过滤器就返回 `True`。过滤器每天针对每支证券返回 `True` 或 `False` 值。然后，将此过滤器添加到管道中，并使用 `.show_graph()` 显示添加该过滤器后的管道图表。

In [ ]:
# Create a Filter that returns True whenever the 15-day average closing price is above $100
high_mean = mean_close_15 > 100

# Add the filter to our pipeline
pipeline.add(high_mean, 'High Mean')

# Render the pipeline as a DAG
pipeline.show_graph()

在上面的图表中，可以清晰地看到我们添加的过滤器。现在我们可以再次运行管道并看看输出。像之前那样运行管道。

In [ ]:
# Set starting and end dates
start_date = pd.Timestamp('2014-01-06', tz='utc')
end_date = pd.Timestamp('2016-01-05', tz='utc')

# Run our pipeline for the given start and end dates
output = engine.run_pipeline(pipeline, start_date, end_date)

# Display the pipeline output
output.head()

可以看到，现在输出 dataframe 包含两列，一列是过滤器，另一列是因子。新列的名称为 `High Mean`，即我们之前为过滤器设定的名称。注意，过滤器列仅包含布尔值，只有 15 天平均收盘价高于 100 美元的证券值为 `True`。